In [1]:
from wav2vec2 import Wav2Vec2Config, Wav2Vec2ForCTC, Wav2Vec2Config

import tensorflow as tf

In [2]:
config = Wav2Vec2Config()
model = Wav2Vec2ForCTC(config)

In [3]:
batch = tf.ones((1, 128, 1), dtype=tf.int32)
model(batch, batch)

InvalidArgumentError: cannot compute Conv2D as input #1(zero-based) was expected to be a int32 tensor but is a float tensor [Op:Conv2D]